In [1]:
import requests
from bs4 import BeautifulSoup

def get_job_links_page(page):
    base_url = "https://www.indeed.com/jobs?"
    params = {'q': 'data scientist',
             'l': 'Denver, CO'}

    # start = "https://www.indeed.com/jobs?q=Data+Scientist&l=Denver%2C+CO"
    # use a fake header
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}
    params['start'] = 10 * (page-1)

    page = requests.get(base_url, params=params, headers=headers)
    # test = requests.get(start, headers=headers)
    
    soup = BeautifulSoup(page.text, "html.parser")
    links = soup.find_all("a")
    
    # build a list of links
    some_links = []

    for l in links:
        try:
            hyperlink = l.attrs.get('href')
            if "/rc/clk?" in hyperlink:
                some_links.append(l.attrs.get('href'))
        except:
            pass
    
    job_links = ["https://www.indeed.com{}".format(x)
             for x in some_links
             ]
    
    return job_links

In [2]:
def get_job_summary(link):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}
    test = requests.get(link, headers=headers)
    soup = BeautifulSoup(test.text, "html.parser")
    spans = soup.find_all('span')
    spans_w_divs = [span.find_all('div') for span in spans if len(span.find_all('div')) > 0]
    span = soup.find("span", id="job_summary")
    return str(span)

In [3]:
import csv
from nltk.tokenize import RegexpTokenizer

def initialize_highlighting(filename):
# Read a list of skill phrases from a file
    skill_phrases = []
    with open(filename, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            skill_phrases.append(str(*row).strip())
    skill_phrases = set(skill_phrases)

    # Write this clean version back out
    with open('skill_phrases_out.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for skill_phrase in skill_phrases:
            csv_writer.writerow([skill_phrase])
        
    #separate each skill phrase into a list of its words
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+-]+')
    skill_phrase_wl = [tokenizer.tokenize(skill_phrase) for skill_phrase in skill_phrases]
    
    return skill_phrase_wl

In [65]:
def tally_skill_mentions_in_job(t, skill_phrase_wl):
    
    skill_mentions_in_job = defaultdict(int)
    # tokenize the text of the description, without spans
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+-]+')
    tokens = tokenizer.tokenize(t)
    # create a dictionary of the words in the job description
    word_index = defaultdict(list)
    for i, k in enumerate(tokens):
        word_index[k].append(i)
    
    # search the word_index dictionary to find the fist word of each skill_phrase
    for skill_phrase in skill_phrase_wl:
        if word_index.get(skill_phrase[0]):
            for occurence in word_index.get(skill_phrase[0]):
                # Check to see if the whole phrase matches
                if all((skill_phrase[j] == tokens[j+occurence]) for j in range(len(skill_phrase))):
                    skill_mentions_in_job[tuple(skill_phrase)] += 1
    return skill_mentions_in_job        

In [70]:
tallied_skill_mentions_in_job = tally_skill_mentions_in_job(t, skill_phrase_wl)

In [4]:
from collections import defaultdict
import pandas as pd
from IPython.core.display import HTML

# requires the global variable skill_phrase_wl
# maybe this should be a parameter rather than a global
def highlight_phrases_from_list(t):

    # tokenize the text of the description, with spans
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+-]+')
    span_generator = tokenizer.span_tokenize(t)
    spans = [span for span in span_generator]
    tokens = [t[span[0]:span[1]] for span in spans]
    
#     # create a dictionary of the words, with spans as the values
#     # and another dictionary with the same keys, with the word indexes as the values
    char_span = defaultdict(list)
    word_index = defaultdict(list)
    for i, (k, span) in enumerate(zip(tokens, spans)):
        char_span[k].append(span)
        word_index[k].append(i)
    # is this useful?
    df = pd.DataFrame({'Character Index Spans': pd.Series(char_span), 'Word Indexes': pd.Series(word_index)})

    highlight_spans = []   
    for skill_phrase in skill_phrase_wl:
        if word_index.get(skill_phrase[0]):
            for i, occurence in enumerate(word_index.get(skill_phrase[0])):
                if all((skill_phrase[j] == tokens[j+occurence]) for j in range(len(skill_phrase))):
                    highlight_span = (spans[occurence][0], spans[occurence + len(skill_phrase) - 1][1])
                    highlight_spans.append (highlight_span)

# # look up the words in our skill list in the dictionary.  List the findings as spans to be highlighted
#     for skill in single_word_skills:
#         highlight_spans += char_span[skill]

    # Sort the spans to be highlighted
    highlight_spans.sort()

    # Insert html tags to highlight the keywords
    html_start_tag = '<font color="red">'
    html_end_tag = '</font>'
    highlighted = ''
    cursor = 0
    for span in highlight_spans:
        if (span[0] > cursor): # go forwards only, not backwards 
            if (cursor>0):
                highlighted += html_end_tag
            highlighted += t[cursor:span[0]] + \
                            html_start_tag + \
                            t[span[0]:span[1]]
        elif (span[1] > cursor):
            highlighted += t[cursor:span[1]]
        cursor = span[1]
    highlighted += html_end_tag + t[cursor:]
    display(HTML(highlighted))    

In [80]:
job_links = []
for page in range(15,16):
    job_links+=(get_job_links_page(page))

In [81]:
from tqdm import tqdm

tallied_skill_mentions = []
skill_phrase_wl = initialize_highlighting('skill_phrases.csv')
skill_dict = {tuple(skill_phrase): 0 for skill_phrase in skill_phrase_wl}
for link in tqdm(job_links):
    # highlight_phrases_from_list(get_job_summary(link))
    tallied_skill_mentions.append(tally_skill_mentions_in_job(get_job_summary(link), skill_dict))

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.30it/s]


In [109]:
df = pd.DataFrame(tallied_skill_mentions, index = job_links).fillna(0).astype(int)

In [110]:
df.columns = [' '.join(c) for c in df.columns.values]

In [111]:
df

,A B,Access,Analysis,B S,Bachelor s Degree,Bachelor s degree,Bachelors,Bayesian,Bioinformatics,C,...,statistical analysis,statistics,technical writing,troubleshooting,visualization,web,web development,work independently,workflow,writing
https://www.indeed.com/rc/clk?jk=89d7e8280b34318e&fccid=5106f86f22f50099&vjs=3,0,0,1,0,0,0,0,0,2,1,...,3,0,0,0,0,0,0,1,1,1
https://www.indeed.com/rc/clk?jk=dbe42786e81109b7&fccid=75aff752794baac1&vjs=3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
https://www.indeed.com/rc/clk?jk=8af06128224c097e&fccid=5609ae8cba643791&vjs=3,1,0,0,0,0,0,0,2,0,0,...,0,3,0,0,0,1,1,0,0,0
https://www.indeed.com/rc/clk?jk=8f053f08ebc069ef&fccid=d7dee2b3330da2cc&vjs=3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
https://www.indeed.com/rc/clk?jk=92cbc1638e2dae62&fccid=e370394a608927d2&vjs=3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.indeed.com/rc/clk?jk=82699a9ab188cd7c&fccid=ad1b508e2719841f&vjs=3,0,1,0,0,0,0,1,0,0,0,...,2,1,0,0,0,0,0,0,0,0
https://www.indeed.com/rc/clk?jk=eeb8e7d5a308d4f7&fccid=bf58a5b90bea7c32&vjs=3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
https://www.indeed.com/rc/clk?jk=82af596a0260f756&fccid=e370394a608927d2&vjs=3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
